In [1]:
import pandas as pd
from sqlalchemy import create_engine, types
from sqlalchemy import text 
from dotenv import dotenv_values

In [2]:
config = dotenv_values()

pg_user = config['POSTGRES_USER'] 
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

In [3]:
engine = create_engine(url, echo=False)

In [4]:
with engine.begin() as conn: 
    result = conn.execute(text(f'SET search_path TO {pg_schema};'))

In [5]:
flights = pd.read_sql(sql=text("SELECT * FROM debby_prep_flights;"), con=engine)

In [6]:
flights.head()

,flight_date,dep_time,sched_dep_time,dep_delay,dep_delay_interval,arr_time,sched_arr_time,arr_delay,arr_delay_interval,airline,...,flight_number,origin,dest,air_time,air_time_interval,actual_elapsed_time,actual_elapsed_time_interval,distance_km,cancelled,diverted
0,2024-07-01,03:07:00,00:49:00,138.0,0 days 02:18:00,08:26:00,06:36:00,110.0,0 days 01:50:00,F9,...,1734,DEN,MCO,179.0,0 days 02:59:00,199.0,0 days 03:19:00,2488.05,0,0
1,2024-07-01,03:24:00,02:49:00,35.0,0 days 00:35:00,06:02:00,05:57:00,5.0,0 days 00:05:00,F9,...,1097,BQN,MCO,138.0,0 days 02:18:00,158.0,0 days 02:38:00,1816.95,0,0
2,2024-07-01,03:02:00,03:11:00,-9.0,-1 days +23:51:00,05:53:00,06:24:00,-31.0,-1 days +23:29:00,F9,...,1823,SJU,MCO,144.0,0 days 02:24:00,171.0,0 days 02:51:00,1913.51,0,0
3,2024-07-01,None,03:30:00,NaN,NaT,None,07:30:00,NaN,NaT,B6,...,746,PSE,JFK,NaN,NaT,NaN,NaT,2602.31,1,0
4,2024-07-01,03:25:00,03:33:00,-8.0,-1 days +23:52:00,06:29:00,06:39:00,-10.0,-1 days +23:50:00,NK,...,796,SJU,MCO,153.0,0 days 02:33:00,184.0,0 days 03:04:00,1913.51,0,0


In [7]:
weather = pd.read_sql(sql=text("SELECT * FROM debby_prep_weather_daily;"), con=engine)

In [8]:
weather.head(3)

,airport_code,station_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,max_snow_mm,avg_wind_direction,avg_wind_speed_kmh,wind_peakgust_kmh,avg_pressure_hpa,sun_minutes,date_day
0,MCO,72205,2024-07-01,27.6,23.3,33.3,0.0,0,174,10.1,None,1015.5,None,1.0
1,TPA,72211,2024-07-01,27.7,25.0,32.2,0.0,0,193,11.9,None,1015.6,None,1.0
2,JFK,74486,2024-07-01,23.3,17.8,27.8,0.0,0,352,24.8,None,1016.2,None,1.0


In [9]:
weather_TPA = weather.query('airport_code == "TPA"')
weather_TPA.head(3)

,airport_code,station_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,max_snow_mm,avg_wind_direction,avg_wind_speed_kmh,wind_peakgust_kmh,avg_pressure_hpa,sun_minutes,date_day
1,TPA,72211,2024-07-01,27.7,25.0,32.2,0.0,0,193,11.9,None,1015.6,None,1.0
4,TPA,72211,2024-07-02,29.4,26.7,33.3,0.0,0,145,9.4,None,1016.7,None,2.0
6,TPA,72211,2024-07-03,29.7,25.0,34.4,6.9,0,76,8.3,None,1017.9,None,3.0


In [10]:
weather_JFK = weather.query('airport_code == "JFK"')
weather_MCO = weather.query('airport_code == "MCO"')

In [11]:
weather_JFK.describe()

,station_id,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,max_snow_mm,avg_wind_direction,avg_wind_speed_kmh,avg_pressure_hpa,date_day
count,62.0,62.000000,62.000000,62.000000,62.000000,62.0,62.000000,62.000000,62.000000,62.000000
mean,74486.0,24.611290,20.983871,28.874194,2.670968,0.0,194.516129,15.869355,1016.209677,16.000000
std,0.0,1.894003,2.356936,2.335958,6.184019,0.0,80.396098,4.269049,4.175169,9.017288
min,74486.0,19.400000,14.400000,22.800000,0.000000,0.0,3.000000,9.400000,1005.700000,1.000000
25%,74486.0,23.600000,19.550000,27.800000,0.000000,0.0,156.750000,13.000000,1013.375000,8.250000
50%,74486.0,24.800000,21.700000,28.900000,0.000000,0.0,194.000000,15.100000,1016.150000,16.000000
75%,74486.0,25.700000,22.200000,30.450000,1.300000,0.0,235.250000,16.900000,1019.075000,23.750000
max,74486.0,28.400000,25.600000,35.000000,32.500000,0.0,352.000000,31.700000,1024.900000,31.000000


In [13]:
flights_MCO = flights.query('dest == "MCO" or origin == "MCO"')
flights_JFK = flights.query('dest == "JFK" or origin == "JFK"')
flights_TPA = flights.query('dest == "TPA" or origin == "TPA"')

In [16]:
import plotly.express as px